## 💻ETL para Movies Dataset 🎥
En el siguiente notebook se presentan las transformaciones requeridas para obtener un dataset limpio y confiable el cual va ser cargado a un modelo de recomendación.


### Importación librerias 📜

Las librerias importadas son las necesarias para realizar con exito el proceso ETL, algunas librerias se eligen debido a las caracteristicas del dataset, por ejemplo el modulo **literal_eval** de la libreria **ast** se usa debido a que los datos de algunas columnas vienen como string pero en realidad representan una estructura de datos más compleja como un dict o un list.

In [153]:
# Importación de librerías requeridas.
import numpy as np 
import pandas as pd
from ast import literal_eval # Importamos este modulo para convertir String a Dict y a List.
import datetime as dt #Esta libreria nos ayuda a trabajar con los campos de tipo fecha.

Ahora leemos los datos desde un archivo CSV usando el metodo read_csv que provee pandas. En este paso hay que tener cuidado con la ruta donde se encuentra almacenado el archivos csv, para este caso accedemos de forma local.

In [154]:
# Usando la función read_csv de Pandas leemos y almacenamos movies_dataset como un dataframe con el nombre movies_data.
movies_data = pd.read_csv('../datasets/movies_dataset.csv', low_memory=False)

Previsualizamos las 5 primeras filas del dataframe así cómo las 5 últimas.

In [155]:
# Con la ayuda del metodo head echamos un vistazo preeliminar al dataframe creado.
movies_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [156]:
# Con la ayuda del metodo tail echamos un vistazo preeliminar al dataframe creado.
movies_data.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


Con el metodo info() echamos un vistazo al dataset completo.

In [157]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

En este punto he tomado la decisión de eliminar las columnas que no aportan información valiosa, en realidad esta decisión tiene sus bases en los requerimientos de más adelante especialmente en el procesamiento de los datos para las funciones y el modelo ML, pero deciso hacerlo de una vez para ir ahorrando procesamiento cuando se ejecuten las transfomaciones siguientes.

In [158]:
# Usando el metodo drop de Pandas borramos las columnas que se consideran inescesarias, se adiciona la opcion inplace=True para que los cambios se reflejen en el mismo dataframe.
movies_data.drop(columns=['video','imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)


### Valores Nulos🗑️
Una de las transformaciones solicitadas, adicional que hace parte fundamental de los ciclos ETL, es el manejo de datos nulos, para nuestro caso debemos cumplir los siguientes criterios:
* Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
* Los valores nulos del campo release date deben eliminarse.

Estas transformaciones se llevan a cabo con el uso de funciones contenidas en pandas especificamente fillna() y dropna().


In [159]:
movies_data['revenue'] = movies_data['revenue'].fillna(0)
movies_data['budget'] = movies_data['budget'].fillna(0)
movies_data['release_date'].dropna(inplace=True)

### Desanidando 🐦

Se procede a desanidar los campos de las columnas 'belongs_to_collection', 'genres', 'production_companies', 'production_countries' y 'spoken_languages'. El metodo usado es el resultado de varios pruebas realizadas con anterioridad para determinar cual es el mejor, inicialmente durante la exploración preeliminar se evidencia que los datos anidados vienen en tipo **str** y con el modulo **literal_eval** de la libreria **ast** se logra convertir a la forma ya sea **dict** o **list**, posteriormente se procede a desanidar y almacenar en dataframes independientes usando el modulo **Series** de Pandas, este metodo se elige ya que se debe cumplir con rapidez con la tarea asignada y no vamos a reinventar la rueda, simplemente usamos lo que ya existe.

In [160]:
# Imprimimos el tipo un campo para cada columna que dice el diccionario de datos que están anidados para visualizar su estructura, luego imprimimos el typo de dato para evidenciar cómo viene formateado.
print(movies_data['belongs_to_collection'][2])
print(movies_data['genres'][2])
print(movies_data['production_companies'][2])
print(movies_data['production_countries'][2])
print(movies_data['spoken_languages'][2])

print(type(movies_data['belongs_to_collection'][2]))
print(type(movies_data['genres'][2]))
print(type(movies_data['production_companies'][2]))
print(type(movies_data['production_countries'][2]))
print(type(movies_data['spoken_languages'][2]))

{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}
[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_639_1': 'en', 'name': 'English'}]
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [161]:
'''Aplicamos una función Lambda a cada columna que viene anidada, dentro de la función lambda se aplica a cada campo la funcion
literal_eval para que trasnforme el string en su estrucutura adecuada ya sea dict o list, en el caso de encontrar un valor nulo se reemplaza con np.nanksk'''

movies_data['belongs_to_collection'] = movies_data['belongs_to_collection'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['genres'] = movies_data['genres'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['production_companies'] = movies_data['production_companies'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['production_countries'] = movies_data['production_countries'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['spoken_languages'] = movies_data['spoken_languages'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)

### Belongs to collection 📚

Para esta columna se evidencia que es un dict anidado por lo tanto vamos a aplicar el metodo **pd.series()**, existen otras herramientas cómo **json_normalize** pero requieren descargar otra librería así que por uso de memoria se descarta esa opción.

In [162]:
# Vamos a aplicar el método pd.series() y vamos a revisar que columnas genera, la serie generada se almacena en una variable independiente.
belongs_to_collection_df = movies_data['belongs_to_collection'].apply(pd.Series)

#Imprimimos las columnas y un campo para revisar que información es relevante
print(belongs_to_collection_df.columns)
print(belongs_to_collection_df[:2])

Index(['id', 'name', 'poster_path', 'backdrop_path', 0], dtype='object')
        id                  name                       poster_path  \
0  10194.0  Toy Story Collection  /7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg   
1      NaN                   NaN                               NaN   

                      backdrop_path   0  
0  /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg NaN  
1                               NaN NaN  


Para este punto se decide conservar sólo la columna 'name' ya que representa un match con alguna secuela o precuela y esa información podría ser relevamte en un sistema de recomendación. Adicionalmente se renombra esa columna para que se diferencie dentrodel data set completo ya que posteriormente procederemos a añadir dicha columna, despues de haber eliminado la original columna 'belongs_to_collection'.

In [163]:
# se usa el metodo drop de pandas para eliminar las columnas no deseadas
belongs_to_collection_df.drop(columns=[0,'id',  'poster_path', 'backdrop_path'], inplace=True)
# Renombrar la columna name con el metodo rename y la pasamos de 'name' a 'collection_name'.
belongs_to_collection_serie = belongs_to_collection_df.rename(columns = {'name' : 'collection_name'})
# Eliminar la columna 'belonngs_to_collection' de movies_data y añadimos la nueva columna 'collction_name'
movies_data.drop('belongs_to_collection', axis=1, inplace=True)
movies_data = pd.concat([movies_data, belongs_to_collection_df], axis=1)


### Genres 🧟‍♂️

En la exploración inicial y en el diccionario de los datos nos muestran que la columna 'genres' viene como una lista de diccionarios ya que una pelicula puede ser clasificada dentro de varios generos, para proceder a desanidarlo lo primero que se realiza es guardar dentro de una variable independiente las columnas generadas al aplicar el metodo explode el  cual nos ayuda a sacar cada elemento de la lista de diccionarios y los almacena en un dataframe, despues aplicamos a ese diccionario el metodo Series como lo hicimos en el caso anterior de 'belongs_to_collection'

In [164]:
#Con el metodo explode() se transforma el dato anidado
genres_df = movies_data['genres'].explode().apply(pd.Series)
genres_df.head()

,id,name,0
0,16.0,Animation,NaN
0,35.0,Comedy,NaN
0,10751.0,Family,NaN
1,12.0,Adventure,NaN
1,14.0,Fantasy,NaN


Ahora vamos a revisar cuales columnas fueron geeneradas para tomar la determinación de cuales podemos conservar.

In [165]:
#Imprimimos las columnas del dataframe creado
genres_df.columns

Index(['id', 'name', 0], dtype='object')

Simplemente se va a conservar la columna name al igual que en el anterior proceso, adicional vamos a proceder con la agrupación de los diferentes generos para cada pelicula, esto se hace aprovechando que  el dataframe conserva el index original extraido al aplicar el metodo Series. Para ello hacemos uso de los metodos join y astype aplicados al dataframe mediante una función lambda, al final se debe resetear el index debido a la caracteristica antes mencionada. el metodo groupby nos ayuda a la agrupación. Como en el caso anterior se renombra la columna para poder unirla al dataset grande.

In [166]:
#Eliminar columnas 0 e Id
genres_df.drop(columns=[0, 'id'], inplace=True)
#Agrupar los generos por el índice original y concatenar los valoresde la columna 'name' en una cadena separada por comas.
genres_df = genres_df.groupby(genres_df.index)['name'].apply(lambda x: ','.join(x.astype(str))).reset_index()
#renombrar columna 'name' por 'genres'
genres_df = genres_df.rename(columns = {'name' : 'genres'})
#almacenar sin el indice
genres_df = genres_df[['genres']]
# Eliminar la columna original 'genres' de movies_data
movies_data.drop('genres', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
movies_data = pd.concat([movies_data, genres_df], axis=1)

### Production Companies 🎬

Se procede de la misma manera que en el caso anterior ya que se tiene una lista de diccionarios.

In [167]:
#Con el metodo explode() se transforma el dato anidado
production_companies_df = movies_data['production_companies'].explode().apply(pd.Series)
production_companies_df.head()

,name,id,0
0,Pixar Animation Studios,3.0,NaN
1,TriStar Pictures,559.0,NaN
1,Teitler Film,2550.0,NaN
1,Interscope Communications,10201.0,NaN
2,Warner Bros.,6194.0,NaN


In [168]:
#Imprimimos las columnas del dataframe creado
production_companies_df.columns

Index(['name', 'id', 0], dtype='object')

Nuevamente dejamos solo la columna 'name'

In [169]:
#Eliminar columnas 0 e Id
production_companies_df.drop(columns=[0, 'id'], inplace=True)
#Agrupar los generos por el índice original y concatenar los valoresde la columna 'name' en una cadena separada por comas.
production_companies_df = production_companies_df.groupby(production_companies_df.index)['name'].apply(lambda x: ','.join(x.astype(str))).reset_index()
#renombrar columna 'name' por 'production_companies'
production_companies_df = production_companies_df.rename(columns = {'name' : 'production_companies'})
#almacenar sin el indice
production_companies_df = production_companies_df[['production_companies']]
# Eliminar la columna original 'production_companies' de movies_data
movies_data.drop('production_companies', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
movies_data = pd.concat([movies_data, production_companies_df], axis=1)

### Production Countries 🗺️

Nuevamente se usa el metodo anterior ya que esta columna presenta la misma estructura de anidación.

In [170]:
#Con el metodo explode() se transforma el dato anidado
production_countries_df = movies_data['production_countries'].explode().apply(pd.Series)
production_countries_df.head()

,iso_3166_1,name,0
0,US,United States of America,NaN
1,US,United States of America,NaN
2,US,United States of America,NaN
3,US,United States of America,NaN
4,US,United States of America,NaN


In [171]:
#Imprimimos las columnas del dataframe creado
production_countries_df.columns

Index(['iso_3166_1', 'name', 0], dtype='object')

In [172]:
#Eliminar columnas 0 e 'iso_3166_1'
production_countries_df.drop(columns=[0, 'iso_3166_1'], inplace=True)
#Agrupar los generos por el índice original y concatenar los valoresde la columna 'name' en una cadena separada por comas.
production_countries_df = production_countries_df.groupby(production_countries_df.index)['name'].apply(lambda x: ','.join(x.astype(str))).reset_index()
#renombrar columna 'name' por 'production_countries'
production_countries_df = production_countries_df.rename(columns = {'name' : 'production_countries'})
#almacenar sin el indice
production_countries_df = production_countries_df[['production_countries']]
# Eliminar la columna original 'production_countries' de movies_data
movies_data.drop('production_countries', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
movies_data = pd.concat([movies_data, production_countries_df], axis=1)

### Spoken Languages 🎌
Nuevamente se usa el metodo anterior ya que esta columna presenta la misma estructura de anidación.

In [173]:
#Con el metodo explode() se transforma el dato anidado
spoken_languages_df = movies_data['spoken_languages'].explode().apply(pd.Series)
spoken_languages_df.head()

,iso_639_1,name,0
0,en,English,NaN
1,en,English,NaN
1,fr,Français,NaN
2,en,English,NaN
3,en,English,NaN


In [174]:
#Eliminar columnas 0 e 'iso_639_1'
spoken_languages_df.drop(columns=[0, 'iso_639_1'], inplace=True)
#Agrupar los generos por el índice original y concatenar los valoresde la columna 'name' en una cadena separada por comas.
spoken_languages_df = spoken_languages_df.groupby(spoken_languages_df.index)['name'].apply(lambda x: ','.join(x.astype(str))).reset_index()
#renombrar columna 'name' por 'spoken_languages'
spoken_languages_df = spoken_languages_df.rename(columns = {'name' : 'spoken_languages'})
#almacenar sin el indice
spoken_languages_df = spoken_languages_df[['spoken_languages']]
# Eliminar la columna original 'spoken_languages' de movies_data
movies_data.drop('spoken_languages', axis=1, inplace=True)
# Concatenar la nueva columna con el DataFrame original
movies_data = pd.concat([movies_data, spoken_languages_df], axis=1)

### Creando la columna return 💰

En este punto se agrega una columna llamada **return** la cual se calcula con los campos **revenue** y **budget**, dividiendo estas dos últimas **revenue / budget**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. En una celda anterior cuando revisamos el tipo de dato de cada columna nos damos cuenta que la columna budget es de tipo object, por lo tanto debemos formatearla antes de proceder, adicional despues de algunas pruebas se evidencia que este campo trae información que no corresponde a un valor numerico represntado con string por lo tanto se usa una función lambda para remplazar esos valores por un '0', usando expresiones regulares ya que durante las pruebas se evidencia que el valor que trae es algo como '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [175]:
movies_data['budget'] = movies_data['budget'].str.replace(r'/[a-zA-Z0-9]+\.jpg', '0', regex=True)
movies_data['budget'] = movies_data['budget'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
movies_data['return'] = movies_data.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

Revisamos la nueva columna creada

In [176]:
print(movies_data['return'].info())
movies_data['return'].head()

<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: return
Non-Null Count  Dtype  
--------------  -----  
45466 non-null  float64
dtypes: float64(1)
memory usage: 355.3 KB
None


0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

### Transformando las fechas 📆

En esta transformación lo que se requiere es lo siguiente:
De haber fechas, deberán tener el formato **AAAA-mm-dd**, además deberán crear la columna **release_year** donde extraerán el año de la fecha de estreno.
Para llevar a cabo esta transformación requerimos el uso de la libreria **datetime**

In [177]:
#Primero inspeccionamos el tipo de dato en el que viene formateado el campo 'release_date'
print(type(movies_data['release_date'][1]))

<class 'str'>


In [178]:
#con el metodo to_datetime de pandas transformamos la str a datetype
movies_data['release_date'] = pd.to_datetime(movies_data['release_date'], errors='coerce' )
#Eliminamos nulos
movies_data = movies_data.dropna(subset=['release_date'])
#Para extraer el año se usa la función year de la libreria datetime
movies_data['release_year'] = movies_data['release_date'].dt.year.astype('int32')

### Manejo de datos duplicados 🧱

Dentro de los procesos ETL es de vital importancia la revisión de data duplicada debido a que esta puede introducir sesgos cuando el dataset final sea cargado para el analisis de algun modelo de ML, por eso y por otras razones dentro de este ETL se realiza este paso como paso final de este ETL y se espera que así de esa forma nuestro modelo de ML tenga una calidad de datos de alimentación muy buena. Dentro de las miles de utilidades de Pandas encontramos df.drop_duplicates() el cual elimina los duplicados, conservando la primera aparición de cada fila especificada dentro de subset.

In [179]:
movies_data = movies_data.drop_duplicates(subset=['id'])

Como ultima medida se debe convertir los datos de la columna 'id' a int ya que estan como str, y al final cuando se unan los dos datasets para alimentar al modelo y a la API será cpn este atributo con el que se haga ese join.

In [180]:
movies_data['id'] = movies_data['id'].astype(int)

In [181]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  int64         
 1   id                    45346 non-null  int64         
 2   original_language     45335 non-null  object        
 3   overview              44405 non-null  object        
 4   popularity            45346 non-null  object        
 5   release_date          45346 non-null  datetime64[ns]
 6   revenue               45346 non-null  float64       
 7   runtime               45100 non-null  float64       
 8   status                45266 non-null  object        
 9   tagline               20387 non-null  object        
 10  title                 45346 non-null  object        
 11  vote_average          45346 non-null  float64       
 12  vote_count            45346 non-null  float64       
 13  name                 

Ahora exporto el dataframe movies_data a un csv para usarlo en el otro ETL para poder unirlos.

In [182]:
# Exportar el DataFrame a un archivo CSV
movies_data.to_csv('../datasets/movies_dataset_final.csv', index=False)  # index=False evita que se exporte el índice